# Variables

## Import des librairies

In [ ]:
import os
import sys
from rdflib import Graph, Namespace, URIRef, Literal, BNode, XSD

## Définition des variables globales

In [ ]:
# Fichiers existants
addr_ont_file_name = "address_ont.ttl"
ev_ont_file_name = "events_ont.ttl"
# ruleset_file_name = "rules.pie"

# Fichiers créés durant le processus
export_file_name = "addresses-temp.ttl"
out_file_name = "addresses.ttl"
local_config_file_name = "config_repo.ttl"

tmp_folder_name = "tmp_files"
mapping_folder_name = "mappings"

project_name = "addresses_from_factoids"
graphdb_url = "http://localhost:7200"
ontorefine_url = "http://localhost:7333"
local_uri = "http://rdf.geohistoricaldata.org/address#"

# Commande pour lancer `Ontorefine`, dépend de l'OS et d'où il se situe
# ontorefine_cmd = "ontorefine-cli"
ontorefine_cmd = "/opt/ontotext-refine/lib/app/bin/ontorefine-cli" #Ubuntu
# ontorefine_cmd = "/Applications/Ontotext\ Refine.app/Contents/app/bin/ontorefine-cli" #MacOS

wd_graph_name = "wikidata"
addr_graph_name = "addresses_from_factoids"

py_code_folder_path = "../code"

## Traitement des variables globales

* Obtention des chemins absolus des fichiers à partir des chemins relatifs donnés dans la section précédente
* Création d'un dossier temporaire s'il n'existe pas encore pour stocker des fichiers à vocation d'être supprimés

In [ ]:
tmp_folder = os.path.abspath(tmp_folder_name)
mapping_folder = os.path.abspath(mapping_folder_name)

python_code_folder = os.path.abspath(py_code_folder_path)

local_config_file = os.path.join(tmp_folder, local_config_file_name)
addr_ont_file = os.path.abspath(addr_ont_file_name)
ev_ont_file = os.path.abspath(ev_ont_file_name)

## Import des modules situés dans le dossier `code`

In [ ]:
# Appel du dossier `code` comprend les codes python
sys.path.insert(1, python_code_folder)

import filemanagement as fm
import wikidata as wd
import ontorefine as otr
import graphdb as gd
import graphrdf as gr
import strprocessing as sp

## Création de dossiers s'ils n'existent pas

In [ ]:
fm.create_folder_if_not_exists(tmp_folder)

## Gestion du répertoire local

### Création du répertoire local dans GraphDB
Pour que la création marche, il faut que GraphDB soit lancé et donc que l'URI donné par `graphdb_url` fonctionne. Si le répertoire existe déjà, rien n'est fait

In [ ]:
# gd.remove_repository(graphdb_url, project_name)
gd.create_config_local_repository_file(local_config_file, project_name)
gd.create_repository_from_config_file(graphdb_url, local_config_file)

### Vidage du répertoire local
Le répertoire dont l'id est `project_name` pour y stocker les données récupérées, cela est utile si le répertoire existait déjà.

In [ ]:
gd.clear_repository(graphdb_url, project_name)

## Import des ontologies

In [ ]:
gd.import_ttl_file_in_graphdb(graphdb_url, project_name, addr_ont_file)
gd.import_ttl_file_in_graphdb(graphdb_url, project_name, ev_ont_file)

# Import des sources

## Voies de Paris via Wikidata

### Variables liées à la source Wikidata

In [ ]:
# `wdpt` pour "wikidata paris thoroughfares"

# Fichier CSV pour stocker le résultat de la requête de la sélection
wdpt_csv_file_name = "wd_paris_thoroughfares.csv"
wdpt_csv_file = os.path.join(tmp_folder, wdpt_csv_file_name)

# Fichier de mapping pour transformer le fichier CSV en fichier TTL
wdpt_mapping_file_name = "mapping_wikidata.json"
wdpt_mapping_file = os.path.join(mapping_folder, wdpt_mapping_file_name)

# Fichier TTL pour structurer les connaissances des voies de Paris
wdpt_kg_file_name = "wd_paris_thoroughfares.ttl"
wdpt_kg_file = os.path.join(tmp_folder, wdpt_kg_file_name)

# Définition des graphes nommés
wdpt_graph_name = "wikidata"
wdpt_ids_graph_name = "wikidata_ids"
wdpt_graph_uri = gd.get_graph_uri_from_name(graphdb_url, project_name, wdpt_graph_name)
wdpt_ids_graph_uri = gd.get_graph_uri_from_name(graphdb_url, project_name, wdpt_ids_graph_name)

### Sélection des voies de Paris sur Wikidata via une requête de sélection

Via une requête SPARQL de sélection, on récupère une liste de ressources à créér :
* pour chaque ressource décrivant une voie dans Paris, on créé autant de ressources qu'elle a de nom officiel. La place de la Nation ([wd:Q1573359](https://www.wikidata.org/entity/Q1573359)) a quatre noms officiels donc on aura 4 ressources. Pour celles qui n'ont pas de nom officiel, on créé une seule ressource dont le nom est le label en français
* ces ressources créées seront liées à celle de Wikidata dont elle provient via `owl:sameAs`
* chaque ressource créée suit la structure définie par l'ontologie
* le résultat de la requête est stocké dans un fichier csv qui sera converti en graphe de connaissance via Ontotext Refine et un fichier de mapping 

In [ ]:
query = f"""
PREFIX addr: <http://rdf.geohistoricaldata.org/address#> 
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
PREFIX pq: <http://www.wikidata.org/prop/qualifier/>
PREFIX ps: <http://www.wikidata.org/prop/statement/>
PREFIX p: <http://www.wikidata.org/prop/>
PREFIX pqv: <http://www.wikidata.org/prop/qualifier/value/>
PREFIX wb: <http://wikiba.se/ontology#>
PREFIX time: <http://www.w3.org/2006/time#>

SELECT *
WHERE {{
    {{
        SELECT DISTINCT * WHERE {{
            {{?voie p:P361 [ps:P361 wd:Q16024163].}}UNION{{?voie p:P361 [ps:P361 wd:Q107311481].}}
            {{?voie p:P1448 ?nomOffSt.
                ?nomOffSt ps:P1448 ?nomOff.
                OPTIONAL {{?nomOffSt pq:P580 ?dateStartStamp; pqv:P580 [wb:timePrecision ?dateStartPrec; wb:timeCalendarModel ?dateStartCal]}}
                OPTIONAL {{?nomOffSt pq:P582 ?dateEndStamp; pqv:P582 [wb:timePrecision ?dateEndPrec; wb:timeCalendarModel ?dateEndCal]}}
            }}UNION{{
                ?voie rdfs:label ?nomOff.
                FILTER (LANG(?nomOff) = "fr")
                MINUS {{?voie p:P1448 ?nomOffSt}}
            }}
        }}
    }}
    BIND(URI(CONCAT(STR(URI(addr:)), "LM_", STRUUID())) AS ?landmark)
    BIND(URI(CONCAT(STR(URI(addr:)), "AN_", STRUUID())) AS ?nameAttribute)
    BIND(URI(CONCAT(STR(URI(addr:)), "ANV_", STRUUID())) AS ?nameAttributeVersion)
    BIND(URI(CONCAT(STR(URI(addr:)), "CH_", STRUUID())) AS ?changeAppearance)
    BIND(URI(CONCAT(STR(URI(addr:)), "EV_", STRUUID())) AS ?eventAppearance)
    BIND(URI(CONCAT(STR(URI(addr:)), "CH_", STRUUID())) AS ?changeDisappearance)
    BIND(URI(CONCAT(STR(URI(addr:)), "EV_", STRUUID())) AS ?eventDisappearance)
    BIND(IF(BOUND(?dateStartStamp), URI(CONCAT(STR(URI(addr:)), "TI", STRUUID())), ?x) AS ?timeInstantAppearance)
    BIND(IF(BOUND(?dateEndStamp), URI(CONCAT(STR(URI(addr:)), "TI", STRUUID())), ?x) AS ?timeInstantDisappearance)
    BIND(IF(?dateStartPrec = 11, time:unitDay, 
                    IF(?dateStartPrec = 10, time:unitMonth,
                        IF(?dateStartPrec = 9, time:unitYear,
                            IF(?dateStartPrec = 8, time:unitDecade,
                                IF(?dateStartPrec = 7, time:unitCentury,
                                    IF(?dateStartPrec = 6, time:unitMillenium, ?x
                                )))))) AS ?dateStartPrecUnit)
    BIND(IF(?dateEndPrec = 11, time:unitDay, 
                    IF(?dateEndPrec = 10, time:unitMonth,
                        IF(?dateEndPrec = 9, time:unitYear,
                            IF(?dateEndPrec = 8, time:unitDecade,
                                IF(?dateEndPrec = 7, time:unitCentury,
                                    IF(?dateEndPrec = 6, time:unitMillenium, ?x
                                )))))) AS ?dateEndPrecUnit)
}}
"""

# wd.save_select_query_as_csv_file(query, wdpt_csv_file)

### Structuration des résultats en graphe de connaissances

In [ ]:
# Si ça ne marche pas ici, c'est sûrement qu'Ontotext Refine n'est pas lancé
otr.get_export_file_from_ontorefine(wdpt_csv_file, wdpt_mapping_file, wdpt_kg_file, ontorefine_cmd, ontorefine_url, project_name)

# Importer le fichier `wdpt_kg_file` qui a été créé lors de la ligne précédente dans le répertoire `project_name`, dans le graphe nommé `wdpt_graph_name` 
gd.import_ttl_file_in_graphdb(graphdb_url, project_name, wdpt_kg_file, wdpt_graph_name)

### Transfert des liens `owl:sameAs` vers un autre graphe nommé

In [ ]:
query = f"""
PREFIX owl: <http://www.w3.org/2002/07/owl#>
DELETE {{
    GRAPH <{wdpt_graph_uri}> {{?s owl:sameAs ?o}}
}}
INSERT {{
    GRAPH <{wdpt_ids_graph_uri}> {{?s owl:sameAs ?o}}
}}
WHERE {{
    GRAPH <{wdpt_graph_uri}> {{?s owl:sameAs ?o}}
}}
"""

gd.update_query(query, graphdb_url, project_name)

## Voies de Paris via les données de la ville de Paris

Import des données de Paris via les données de la ville de Paris :
* [dénominations des emprises des voies actuelles](https://opendata.paris.fr/explore/dataset/denominations-emprises-voies-actuelles)
* [dénominations caduques des voies](https://opendata.paris.fr/explore/dataset/denominations-des-voies-caduques)

Le travail se fait sur deux fichiers csv issus des deux jeux de données.

Chaque ligne des deux fichiers constitue une ressource dans le répertoire. Les voies actuelles ont une emprise géométrique contrairement aux anciennes voies.

### Variables liées aux données de la ville de Paris

In [ ]:
# `vpt` pour "ville paris thoroughfares"

# Fichier CSV pour stocker le résultat de la requête de la sélection
vpta_csv_file_name = "denominations-emprises-voies-actuelles.csv"
vpta_csv_file = os.path.join(tmp_folder, vpta_csv_file_name)
vptc_csv_file_name = "denominations-des-voies-caduques.csv"
vptc_csv_file = os.path.join(tmp_folder, vptc_csv_file_name)

# Fichier de mapping pour transformer le fichier CSV en fichier TTL
vpta_mapping_file_name = "mapping_voies_paris_actuelles.json"
vpta_mapping_file = os.path.join(mapping_folder, vpta_mapping_file_name)
vptc_mapping_file_name = "mapping_voies_paris_caduques.json"
vptc_mapping_file = os.path.join(mapping_folder, vptc_mapping_file_name)

# Fichier TTL pour structurer les connaissances des voies de Paris
vpta_kg_file_name = "voies_paris_actuelles.ttl"
vpta_kg_file = os.path.join(tmp_folder, vpta_kg_file_name)
vptc_kg_file_name = "voies_paris_caduques.ttl"
vptc_kg_file = os.path.join(tmp_folder, vptc_kg_file_name)

# Définition des graphes nommés
vpt_graph_name = "ville_de_paris"
vpt_graph_uri = gd.get_graph_uri_from_name(graphdb_url, project_name, vpt_graph_name)

### Structuration des résultats en graphe de connaissances

In [ ]:
# Si ça ne marche pas ici, c'est sûrement qu'Ontotext Refine n'est pas lancé
otr.get_export_file_from_ontorefine(vpta_csv_file, vpta_mapping_file, vpta_kg_file, ontorefine_cmd, ontorefine_url, project_name)
otr.get_export_file_from_ontorefine(vptc_csv_file, vptc_mapping_file, vptc_kg_file, ontorefine_cmd, ontorefine_url, project_name)

# Importer le fichier `wdpt_kg_file` qui a été créé lors de la ligne précédente dans le répertoire `project_name`, dans le graphe nommé `wdpt_graph_name` 
gd.import_ttl_file_in_graphdb(graphdb_url, project_name, vpta_kg_file, vpt_graph_name)
gd.import_ttl_file_in_graphdb(graphdb_url, project_name, vptc_kg_file, vpt_graph_name)

### Suppresion des instants temporels sans timeStamp

In [ ]:
query = f"""
PREFIX addr: <http://rdf.geohistoricaldata.org/address#>
DELETE {{
    ?timeInstant ?p ?o.
    ?s ?p ?timeInstant.
}}WHERE {{
    GRAPH <{vpt_graph_uri}> {{
        ?timeInstant a addr:TimeInstant.
        MINUS {{?timeInstant addr:timeStamp ?timeStamp}}
        {{?timeInstant ?p ?o}}UNION{{?s ?p ?timeInstant}}
    }}
}}
"""

gd.update_query(query, graphdb_url, project_name)